<hr style="border:1px solid gray"> </hr>

# Monodromy 

*Documentation*: https://www.juliahomotopycontinuation.org/HomotopyContinuation.jl/stable/monodromy/

An alternative to using the `solve` function is solving a polynomial system $F=(f_1,\ldots,f_n)$ by monodromy. 

Here is the basic idea:


Suppose $x$ is a solution $F(x)=0$ and that $F(x)=F(x,u)$ is a point in a family of polynomial systems 

$$P=\{F(x,u) \mid u\in \mathbb{C}^k\}.$$

The monodromy method consists in moving around $u$ in a loop starting and ending at $u_0$ while tracking $x$ along that loop. After one iteration usually one has found a new solution $y\in \mathbb{C}^n$. This process then is repeated until some stopping criterion is fulfilled.


Here is a simple example: take

$$F(x,y,u_1,u_2) = \begin{bmatrix} x^4 + y - 2u_1\\\ x^4 + x^2 - 2u_2y^2 \end{bmatrix}.$$

For the initial parameters $u_0=(1,1)$ we have the solution $(x,y) = (1,1)$. For finding all solutions of $F(x,y,1,1)$ we use

In [ ]:
using HomotopyContinuation
@var x y 
@var u[1:2] # u are the parameters
F = System([x^4 + y - 2u[1], x^4 + x^2 - 2*u[2]*y^2],
            variables = [x;y],
            parameters = u)

In [ ]:
initial_solutions = [[1; 1]]
initial_parameter = [1, 1]
monodromy_solve(F, initial_solutions, initial_parameter)

It is possible to not pass initial solution and initial parameters. 

In this case, the software will generate random complex parameters and try to compute one solutions with Newton's method.

In [ ]:
S = monodromy_solve(F)

In [ ]:
parameters(S)

<hr style="border:1px solid gray"> </hr>

# Group Actions

If the set of solutions of `F` is invariant under some group actions you can exploit this in your computation.


In the above example, the group that interchanges `x` and `y` acts on the solution set of `F`. We can use the group that multiplies `x` by $\pm 1$.

In [ ]:
G = GroupActions( a -> ([-a[1], a[2]], ))
monodromy_solve(F, 
                initial_solutions, 
                initial_parameter, 
                group_actions = G)

Now, we found only 4 solutions: one from each orbit. If we suppress computing with equivalence classes, then

In [ ]:
monodromy_solve(F, 
                initial_solutions, 
                initial_parameter, 
                group_actions = G, 
                equivalence_classes = false)

This also works:

In [ ]:
monodromy_solve(F, group_actions = G)

<hr style="border:1px solid gray"> </hr>

# Permutations

Monodromy loops incude a permutations on the zeros of $F=0$. 

They can be extracted as follows:

In [ ]:
S = monodromy_solve(F; permutations = true) # the default is `permutations = false`

In [ ]:
permutations(S)

This table shows the permuted indices in each column. 

The columns represent different loops.

The table does not show repeated permutations. 

If you want a table that shows in the $i$-th column the permutations induced by the $i$-th loop, then this works:

In [ ]:
permutations(S; reduced = false)